In [2]:
from datasets import load_dataset, load_metric
device = 'cuda'

# Dataset is downloaded from https://www.openslr.org/12
data = load_dataset("LibriSpeech")

Resolving data files:   0%|          | 0/585 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/87 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/97 [00:00<?, ?it/s]

Using custom data configuration LibriSpeech-49d9aec4ade7f44e
Reusing dataset text (/home/kmb85/.cache/huggingface/datasets/text/LibriSpeech-49d9aec4ade7f44e/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# from parse_data import parse_to_json
import json
import os

# parse_to_json("./LibriSpeech/dev-clean")
# os.rename('data.json', 'dev-clean.json')
with open('dev-clean.json', 'r') as file:
    dev_clean_data = json.load(file)

# parse_to_json("./LibriSpeech/test-clean")
# os.rename('data.json', 'test-clean.json')
with open('test-clean.json', 'r') as file:
    test_clean_data = json.load(file)

In [4]:
dev_clean_dict = {}
for key, value in dev_clean_data.items():
    new_key = key.replace('-', '')
    dev_clean_dict[new_key] = value

In [5]:
test_clean_dict = {}
for key, value in test_clean_data.items():
    new_key = key.replace('-', '')
    test_clean_dict[new_key] = value

In [6]:
import soundfile as sf
import resampy

def flac_to_array_with_sampling_rate(file_path, target_sr=16000):
    data, sr = sf.read(file_path, dtype='float32')

    if sr != target_sr:
        data = resampy.resample(data, sr, target_sr)

    return data

In [7]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters_dev(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    trimmed_string = batch["text"].strip()
    words = trimmed_string.split()
    new_text = ' '.join(words[1:])
    batch["text"] = new_text
    batch["input_values"] = flac_to_array_with_sampling_rate(dev_clean_dict[words[0]]['file_path'])
    return batch

In [8]:
data['validation'] = data['validation'].map(remove_special_characters_dev)

Parameter 'function'=<function remove_special_characters_dev at 0x7fdf403fbb50> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

In [9]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters_test(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    trimmed_string = batch["text"].strip()
    words = trimmed_string.split()
    new_text = ' '.join(words[1:])
    batch["text"] = new_text
    batch["input_values"] = flac_to_array_with_sampling_rate(test_clean_dict[words[0]]['file_path'])
    return batch

In [10]:
data['test'] = data['test'].map(remove_special_characters_test)

0ex [00:00, ?ex/s]

In [11]:
from speechbrain.pretrained import EncoderASR
import torch

In [12]:
asr_model = EncoderASR.from_hparams(source="speechbrain/asr-wav2vec2-librispeech", savedir="pretrained_models/asr-wav2vec2-librispeech", run_opts={"device":"cuda"})

hyperparams.yaml:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

/home/kmb85/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model ch

wav2vec2.ckpt:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

asr.ckpt:   0%|          | 0.00/8.53M [00:00<?, ?B/s]

tokenizer.ckpt:   0%|          | 0.00/426 [00:00<?, ?B/s]

In [19]:
import pickle

def get_activation(name):
    def hook(model, input, output):
        activation_outputs[name] = output.detach().cpu()
    return hook

def get_input_activation(name):
    def hook(model, input, output):
        # Check if the activation for this layer has already been started
        if name not in activation_inputs:
            activation_inputs[name] = []
        # Append the input activations for this batch
        activation_inputs[name].append(input[0].detach().cpu())
    return hook

def save_activations(activations, filepath):
    concatenated_activations = {}

    for name, acts in activations.items():
        # The size of the last tensor in each list
        max_size = acts[-1].size()

        # Pad each tensor in the list to have the same size as the last tensor
        padded_acts = []
        for act in acts:
            # Initialize padding list
            pad = []

            # Calculate padding for each dimension, except the first (batch) dimension
            for dim in range(1, act.dim()):
                padding = max_size[dim] - act.size(dim)
                pad = [0, padding] + pad  # Padding is added as [before, after] for each dimension

            # Apply padding and store the result
            padded_act = torch.nn.functional.pad(act, pad)
            padded_acts.append(padded_act)

        # Concatenate all the padded tensors along dimension 0
        concatenated_activations[name] = torch.cat(padded_acts, dim=0)

        # Save the concatenated activation to a file
        filepath = f'{name}.pkl'
        with open(filepath, 'wb') as f:
            pickle.dump(concatenated_activations[name], f)
        print(f'Saved {filepath}')

In [15]:
layers = [asr_model.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv]
names = ['conv_0']
activation_inputs = {}
for layer, name in zip(layers, names):
        layer.register_forward_hook(get_input_activation(name))

In [20]:
def map_model_to_wer_result(model, single_batch):
    input_values = torch.tensor(single_batch["input_values"]).to('cuda')
    with torch.no_grad():
        prediction = model.transcribe_batch(input_values, torch.tensor([1.0]).to('cuda'))

num_samples = 100
for i in range(num_samples):
    map_model_to_wer_result(asr_model, data['test'][i:i+1])


In [21]:
save_activations(activation_inputs, 'input_activations_cnn_dnn.pkl')

Saved conv_0.pkl
